# Get Jurisdiction Demographics

* About ACS: https://www.census.gov/data/developers/data-sets/acs-5year.html
* Census API documentation: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
* Uses [`census`](https://pypi.org/project/census/) Python library to get ACS data (median household income, % people of color, and % cost-burdened households) for relevant geographies. May need to run `pip install census` to install the library prior to running this notebook.
* Detailed profiles: https://api.census.gov/data/2020/acs/acs5/profile/variables.html
* Regular tables: https://api.census.gov/data/2020/acs/acs5/variables.html

In [88]:
import pandas as pd
from census import Census

In [89]:
# Initialize Census with API key
c = Census('814fa83c203ce788d409c12321970a9cf0bf1b03')

# Define geographic level
geo = {
    'for': 'county subdivision:*',
    'in': 'state:09'
}

year = 2019 # ACS release year

### Get median household income per jurisdiction

In [90]:
# Get data for all towns (county subdvisions) in CT (state code 09)
income_raw = c.acs5.get(
    ['NAME', 'B19013_001E'], # variables
    geo,
    year=year # ACS release year
)

income = pd.DataFrame(
    income_raw
).rename(
    columns={
        'B19013_001E': 'income'
    }
).filter(['NAME', 'income'])

income.head()

,NAME,income
0,"Westbrook town, Middlesex County, Connecticut",79707.0
1,"County subdivisions not defined, Middlesex Cou...",-666666666.0
2,"East Haddam town, Middlesex County, Connecticut",96928.0
3,"Old Saybrook town, Middlesex County, Connecticut",83132.0
4,"Durham town, Middlesex County, Connecticut",119905.0


### Calculate % BIPOC per jurisdiction

In [91]:
bipoc_raw = c.acs5dp.get(
    ['NAME', 'DP05_0077PE'],
     geo,
     year=year
)

bipoc = pd.DataFrame(
    bipoc_raw
).assign(
    bipoc=lambda df_: round(100 - df_.DP05_0077PE, 1)
).filter(['NAME', 'bipoc'])

bipoc.head()

,NAME,bipoc
0,"Westbrook town, Middlesex County, Connecticut",7.9
1,"County subdivisions not defined, Middlesex Cou...",666666766.0
2,"East Haddam town, Middlesex County, Connecticut",4.8
3,"Old Saybrook town, Middlesex County, Connecticut",10.5
4,"Durham town, Middlesex County, Connecticut",3.5


### Calculate % cost-burdened households per jurisdiction

In [92]:
costburdened_raw = c.acs5dp.get(
    ['NAME',
     # owners, with mortgage:
     'DP04_0110E', 'DP04_0114E', 'DP04_0115E',
     # owners, without mortgage:
     'DP04_0117E', 'DP04_0123E', 'DP04_0124E',
     # renters:
     'DP04_0136E', 'DP04_0141E', 'DP04_0142E'
    ],
    geo,
    year=year
)

costburdened = pd.DataFrame(
    costburdened_raw
).assign(
    num=lambda df_: df_.DP04_0114E + df_.DP04_0115E \
        + df_.DP04_0123E + df_.DP04_0124E + df_.DP04_0141E + df_.DP04_0142E,
    denom=lambda df_: df_.DP04_0110E + df_.DP04_0117E + df_.DP04_0136E,
    burdened=lambda df_: round(df_.num / df_.denom*100, 1)
).filter(['NAME', 'burdened'])

costburdened.head()

,NAME,burdened
0,"Westbrook town, Middlesex County, Connecticut",37.1
1,"County subdivisions not defined, Middlesex Cou...",NaN
2,"East Haddam town, Middlesex County, Connecticut",26.2
3,"Old Saybrook town, Middlesex County, Connecticut",33.3
4,"Durham town, Middlesex County, Connecticut",21.6


### Combine three variables into a single dataframe, and save

In [93]:
demographics = (income
                .merge(bipoc, on='NAME')
                .merge(costburdened, on='NAME')
                .assign(
                    NAME=lambda df_: df_.NAME.apply(lambda x: x.split(' town,')[0])
                )
               )

# Save to file
with open('./demographics.js', 'w') as f:
    f.write(
        'const demographics = ' +\
        str(demographics[~demographics.NAME.str.startswith('County')]
         .set_index('NAME')
         .to_dict('index')
        )
    )